# Bibliotheken

In [72]:
import pandas as pd
import json
import networkx as nx
import re
from sklearn.metrics.pairwise import cosine_similarity

In [73]:
#!pip install sklearn

# Daten einlesen

In [95]:
namen = pd.read_excel("namen.xlsx")
namen.index = namen["CouncillorId"]
namen = namen.T
namen = namen[1:]
namen.head()

CouncillorId,CouncillorBioId,CouncillorName,Rat,Fraktion,Kanton,Geburtsdatum,Vereidigungsdatum,nan
3055,4154,Addor Jean-Luc,NR,V,VS,1964-04-22,2015-11-30,4154
2670,3867,Aebi Andreas,NR,V,BE,1958-11-26,2015-11-30,3867
2760,4049,Aebischer Matthias,NR,S,BE,1967-10-18,2015-11-30,4049
2758,4053,Aeschi Thomas,NR,V,ZG,1979-01-13,2015-11-30,4053
2796,4090,Amaudruz Céline,NR,V,GE,1979-03-15,2015-11-30,4090


In [80]:
df = pd.read_excel("parlament_export_session_5019_de_CH.xlsx")
df = df.drop(['Decision','Ja', "Nein", "Enth.", "Entschuldigung gem. Art. 57 Abs. 4", "Hat nicht teilgenommen", "Präsident"], axis=1)
df.head()

,VoteDate,Rat,Kommission,Dept.,AffairId,AffairTitle,VoteRegistrationNumber,VoteMeaningYes,VoteMeaningNo,DivisionText,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
0,2019-09-09 15:26:55,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19203,Accettare l'articolo,Respingere l'articolo,NaN,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
1,2019-09-09 15:27:54,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19204,Accettare il progetto,Respingere il progetto,Votazione sul complesso,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
2,2019-09-09 16:16:12,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19207,Proposta della maggioranza (entrare in materia),Proposta della minoranza Imark (non entrare in...,Entrare in materia,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein,Nein
3,2019-09-09 17:33:42,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19216,Proposta della maggioranza e Consiglio federale,Proposta della minoranza Müller-Altermatt,"Cifra II, Legge sul CO2, art. 3 cpv. 1bis",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja
4,2019-09-09 17:34:46,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19209,Proposta della maggioranza e Consiglio federal...,Proposta della minoranza Girod,"Cifra II, Legge sul CO2, art. 10 cpv. 1bis e 2...",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja


# Index Setzen

In [81]:
df.index = df["AffairTitle"]
df.head()

,VoteDate,Rat,Kommission,Dept.,AffairId,AffairTitle,VoteRegistrationNumber,VoteMeaningYes,VoteMeaningNo,DivisionText,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
AffairTitle,,,,,,,,,,,,,,,,,,,,,
Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,2019-09-09 15:26:55,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19203,Accettare l'articolo,Respingere l'articolo,NaN,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,2019-09-09 15:27:54,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19204,Accettare il progetto,Respingere il progetto,Votazione sul complesso,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 16:16:12,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19207,Proposta della maggioranza (entrare in materia),Proposta della minoranza Imark (non entrare in...,Entrare in materia,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein,Nein
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 17:33:42,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19216,Proposta della maggioranza e Consiglio federale,Proposta della minoranza Müller-Altermatt,"Cifra II, Legge sul CO2, art. 3 cpv. 1bis",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 17:34:46,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19209,Proposta della maggioranza e Consiglio federal...,Proposta della minoranza Girod,"Cifra II, Legge sul CO2, art. 10 cpv. 1bis e 2...",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja


# Tabelle 90 Grad Kippen

In [82]:
df_t = df.T
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
VoteDate,2019-09-09 15:26:55,2019-09-09 15:27:54,2019-09-09 16:16:12,2019-09-09 17:33:42,2019-09-09 17:34:46,2019-09-09 17:36:08,2019-09-09 17:37:27,2019-09-09 17:38:25,2019-09-09 17:39:22,2019-09-09 17:40:54,...,2019-09-27 09:23:53,2019-09-27 09:24:41,2019-09-27 09:44:37,2019-09-27 09:45:36,2019-09-27 09:46:26,2019-09-27 09:47:14,2019-09-27 09:48:23,2019-09-27 09:49:33,2019-09-27 09:50:25,2019-09-27 09:54:11
Rat,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR,...,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR
Kommission,"UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR",...,"RK-NR, WBK-SR","RK-NR, RK-SR","WAK-NR, WAK-SR","WBK-NR, SGK-SR","SGK-NR, SGK-SR","UREK-NR, UREK-SR","WBK-NR, SGK-SR","APK-NR, APK-SR","SiK-NR, SiK-SR","SPK-NR, SPK-SR"
Dept.,UVEK,UVEK,Parl,Parl,Parl,Parl,Parl,Parl,Parl,Parl,...,EJPD,EJPD,EFD,EDI,EDI,UVEK,EDI,EJPD,VBS,WBF
AffairId,18.097,18.097,17.405,17.405,17.405,17.405,17.405,17.405,17.405,17.405,...,17.069,18.049,18.05,18.052,18.091,18.095,18.441,19.017,19.022,19.024


# Ersten Infofelder übergehen

In [83]:
df_t = df_t[12:]
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
4154,Enthaltung,Enthaltung,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Enthaltung,Ja,Ja,Ja,Nein,Ja,Nein
3867,Ja,Ja,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Enthaltung,Ja,Ja,Nein
4049,Ja,Ja,Ja,Nein,Nein,Nein,Nein,Nein,Nein,Nein,...,Ja,Nein,Nein,Nein,Ja,Ja,Ja,Ja,Enthaltung,Ja
4053,Ja,Ja,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Nein,Ja,Nein
4090,Hat nicht teilgenommen,Ja,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein


# Kategorien durch Zahlen ersetzen

In [7]:
cleanup_dict = {"Ja": 1, "Nein": 0, "Hat nicht teilgenommen": 2, "Enthaltung": 3, "Der Präsident stimmt nicht": 4, "Entschuldigt": 5}

In [84]:
df_t.replace(cleanup_dict, inplace=True)
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
4154,3,3,0,1,1,1,1,1,1,1,...,1,1,1,3,1,1,1,0,1,0
3867,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,3,1,1,0
4049,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,3,1
4053,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,0
4090,2,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,0


# Cosine Similarity berechnen

In [88]:
pd.DataFrame(cosine_similarity([[1,0,1,0],[0,0,0,0],[1,0,1,1]]))

,0,1,2
0,1.000000,0.0,0.816497
1,0.000000,0.0,0.000000
2,0.816497,0.0,1.000000


In [91]:
dists = cosine_similarity(df_t)
dists_named = pd.DataFrame(dists, columns=df_t.index)
dists_named.index = df_t.index
dists_named

,4154,3867,4049,4053,4090,4180,1135,4155,4184,4058,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
4154,1.000000,0.756431,0.258163,0.828215,0.814113,0.622053,0.720019,0.752536,0.249096,0.304761,...,0.625717,0.598232,0.250140,0.563890,0.496198,0.380822,0.759075,0.252568,0.839167,0.772511
3867,0.756431,1.000000,0.337528,0.805108,0.814318,0.649850,0.724418,0.801022,0.322557,0.393208,...,0.659738,0.620713,0.311465,0.599370,0.480401,0.407769,0.822673,0.318136,0.851318,0.828375
4049,0.258163,0.337528,1.000000,0.240249,0.300094,0.611089,0.355398,0.303642,0.804290,0.763609,...,0.521232,0.571150,0.785549,0.500480,0.614317,0.704155,0.296542,0.826541,0.245754,0.291561
4053,0.828215,0.805108,0.240249,1.000000,0.845575,0.654395,0.790120,0.827821,0.203407,0.302671,...,0.649266,0.627308,0.226517,0.530101,0.443557,0.336972,0.849730,0.223438,0.937124,0.854352
4090,0.814113,0.814318,0.300094,0.845575,1.000000,0.661734,0.739741,0.792754,0.273760,0.330106,...,0.688241,0.651707,0.299998,0.622005,0.460538,0.393029,0.818637,0.313495,0.891853,0.863060
4180,0.622053,0.649850,0.611089,0.654395,0.661734,1.000000,0.675961,0.646705,0.562826,0.650987,...,0.762338,0.752293,0.576342,0.661536,0.698936,0.628127,0.642966,0.603509,0.650994,0.649865
1135,0.720019,0.724418,0.355398,0.790120,0.739741,0.675961,1.000000,0.706704,0.360404,0.460479,...,0.664170,0.634148,0.347953,0.546252,0.526942,0.458887,0.747624,0.388086,0.821141,0.743565
4155,0.752536,0.801022,0.303642,0.827821,0.792754,0.646705,0.706704,1.000000,0.314695,0.372236,...,0.639159,0.714104,0.310371,0.523635,0.463547,0.374814,0.801228,0.288154,0.864220,0.829389
4184,0.249096,0.322557,0.804290,0.203407,0.273760,0.562826,0.360404,0.314695,1.000000,0.808115,...,0.510662,0.561896,0.841807,0.487669,0.612848,0.767980,0.264524,0.859272,0.210553,0.258438
4058,0.304761,0.393208,0.763609,0.302671,0.330106,0.650987,0.460479,0.372236,0.808115,1.000000,...,0.548618,0.588910,0.792250,0.478791,0.623603,0.744305,0.319592,0.804896,0.294406,0.337600


In [92]:
dists = cosine_similarity(df_t)
dists_named = pd.DataFrame(dists, columns=df_t.index)
dists_named.index = df_t.index
filtered = dists_named[dists_named > 0.5]
filtered = filtered.fillna(0)
filtered

,4154,3867,4049,4053,4090,4180,1135,4155,4184,4058,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
4154,1.000000,0.756431,0.000000,0.828215,0.814113,0.622053,0.720019,0.752536,0.000000,0.000000,...,0.625717,0.598232,0.000000,0.563890,0.000000,0.000000,0.759075,0.000000,0.839167,0.772511
3867,0.756431,1.000000,0.000000,0.805108,0.814318,0.649850,0.724418,0.801022,0.000000,0.000000,...,0.659738,0.620713,0.000000,0.599370,0.000000,0.000000,0.822673,0.000000,0.851318,0.828375
4049,0.000000,0.000000,1.000000,0.000000,0.000000,0.611089,0.000000,0.000000,0.804290,0.763609,...,0.521232,0.571150,0.785549,0.500480,0.614317,0.704155,0.000000,0.826541,0.000000,0.000000
4053,0.828215,0.805108,0.000000,1.000000,0.845575,0.654395,0.790120,0.827821,0.000000,0.000000,...,0.649266,0.627308,0.000000,0.530101,0.000000,0.000000,0.849730,0.000000,0.937124,0.854352
4090,0.814113,0.814318,0.000000,0.845575,1.000000,0.661734,0.739741,0.792754,0.000000,0.000000,...,0.688241,0.651707,0.000000,0.622005,0.000000,0.000000,0.818637,0.000000,0.891853,0.863060
4180,0.622053,0.649850,0.611089,0.654395,0.661734,1.000000,0.675961,0.646705,0.562826,0.650987,...,0.762338,0.752293,0.576342,0.661536,0.698936,0.628127,0.642966,0.603509,0.650994,0.649865
1135,0.720019,0.724418,0.000000,0.790120,0.739741,0.675961,1.000000,0.706704,0.000000,0.000000,...,0.664170,0.634148,0.000000,0.546252,0.526942,0.000000,0.747624,0.000000,0.821141,0.743565
4155,0.752536,0.801022,0.000000,0.827821,0.792754,0.646705,0.706704,1.000000,0.000000,0.000000,...,0.639159,0.714104,0.000000,0.523635,0.000000,0.000000,0.801228,0.000000,0.864220,0.829389
4184,0.000000,0.000000,0.804290,0.000000,0.000000,0.562826,0.000000,0.000000,1.000000,0.808115,...,0.510662,0.561896,0.841807,0.000000,0.612848,0.767980,0.000000,0.859272,0.000000,0.000000
4058,0.000000,0.000000,0.763609,0.000000,0.000000,0.650987,0.000000,0.000000,0.808115,1.000000,...,0.548618,0.588910,0.792250,0.000000,0.623603,0.744305,0.000000,0.804896,0.000000,0.000000


# Als Netzwerk exportieren

In [98]:
namen_dict = dict(zip(namen["CouncillorBioId"],namen["CouncillorName"]))
fract_dict = dict(zip(namen["CouncillorBioId"],namen["Fraktion"]))
fract_dict

{4154: 'V',
 3867: 'V',
 4049: 'S',
 4053: 'V',
 4090: 'V',
 4180: 'C',
 1135: 'V',
 4155: 'V',
 4184: 'G',
 4058: 'S',
 4138: 'C',
 4203: 'S',
 4187: 'RL',
 1138: 'GL',
 4182: 'C',
 4213: 'S',
 4102: 'GL',
 4209: 'RL',
 4030: 'S',
 4188: 'RL',
 3872: 'RL',
 4067: 'V',
 4230: 'C',
 724: 'G',
 4215: 'RL',
 4025: 'V',
 4156: 'V',
 4157: 'V',
 4101: 'C',
 4158: 'V',
 4189: 'RL',
 4200: 'BD',
 4070: 'C',
 3830: 'S',
 4217: 'RL',
 4108: 'GL',
 4159: 'V',
 4135: 'V',
 4219: 'S',
 1124: 'C',
 4051: 'V',
 4201: 'G',
 4079: 'RL',
 4160: 'V',
 4190: 'RL',
 4229: 'V',
 4212: 'C',
 4110: 'V',
 3875: 'RL',
 3876: 'V',
 74: 'RL',
 4195: 'S',
 4077: 'RL',
 4069: 'S',
 3877: 'RL',
 4059: 'GL',
 3878: 'V',
 1122: 'RL',
 4042: 'V',
 4227: 'GL',
 4074: 'S',
 4131: 'S',
 3880: 'V',
 4191: 'RL',
 91: 'V',
 3882: 'G',
 1337: 'C',
 4161: 'V',
 4093: 'G',
 3883: 'V',
 4096: 'C',
 4181: 'C',
 4137: 'V',
 4116: 'RL',
 806: 'G',
 1295: 'S',
 3885: 'V',
 4083: 'GL',
 3886: 'BD',
 4162: 'V',
 4087: 'C',
 4214: 'C'

In [99]:
name_mappings

{0: 'Addor Jean-Luc',
 1: 'Aebi Andreas',
 2: 'Aebischer Matthias',
 3: 'Aeschi Thomas',
 4: 'Amaudruz Céline',
 5: 'Ammann Thomas',
 6: 'Amstutz Adrian',
 7: 'Arnold Beat',
 8: 'Arslan Sibel',
 9: 'Badran Jacqueline',
 10: 'Barazzone Guillaume',
 11: 'Barrile Angelo',
 12: 'Bauer Philippe',
 13: 'Bäumle Martin',
 14: 'Béglé Claude',
 15: 'Bendahan Samuel',
 16: 'Bertschy Kathrin',
 17: 'Bigler Hans-Ulrich',
 18: 'Birrer-Heimo Prisca',
 19: 'Borloz Frédéric',
 20: 'Bourgeois Jacques',
 21: 'Brand Heinz',
 22: 'Bregy Philipp Matthias',
 23: 'Brélaz Daniel',
 24: 'Brunner Hansjörg',
 25: 'Büchel Roland Rino',
 26: 'Buffat Michaël',
 27: 'Bühler Manfred',
 28: 'Bulliard-Marbach Christine',
 29: 'Burgherr Thomas',
 30: 'Burkart Thierry',
 31: 'Campell Duri',
 32: 'Candinas Martin',
 33: 'Carobbio Guscetti Marina',
 34: 'Cattaneo Rocco',
 35: 'Chevalley Isabelle',
 36: 'Chiesa Marco',
 37: 'Clottu Raymond',
 38: 'Crottaz Brigitte',
 39: 'de Buman Dominique',
 40: 'de Courten Thomas',
 41: '

In [46]:
namen_dict = dict(zip(namen["CouncillorBioId"],namen["CouncillorName"]))
fract_dict = dict(zip(namen["CouncillorBioId"],namen["Fraktion"]))
name_mappings = {}
fract_mappings = {}
i = 0
for id in df_t.index:
    name_mappings[i] = namen_dict[id]
    fract_mappings[i] = fract_dict[id]
    i += 1

In [102]:
G=nx.from_numpy_matrix(filtered.values)
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 199\nNumber of edges: 15262\nAverage degree: 153.3869'

In [103]:
G=nx.from_numpy_matrix(filtered.values)
nx.set_node_attributes(G, values = fract_mappings, name='fraktion')
H = nx.relabel_nodes(G, name_mappings)
nx.write_gexf(H,"Politics.gexf")